# Import demo dataframe

Since the *All of Us* data is restricted, we have provided a demo data frame with fake data to test the code for the models

We now import the relevant dataframe and rename it `birth`

In [ ]:
# Run as needed
!pip install import-ipynb

In [ ]:
import import_ipynb
print("Thanks for your patience while I import the dataframe.")
from rawlifestyledataframe import *
print("All done.")

In [ ]:
birth = df

# Models

## Import Packages

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import recall_score, precision_score, f1_score, precision_recall_curve, auc

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.svm import LinearSVC

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures

## Binarize Response Variable

In [ ]:
birth['birth_class_binary'] = 0
birth.loc[birth.birth_class=='Preterm', 'birth_class_binary'] = 1

In [ ]:
birth.columns

## Split data into training and testing data

We will use a stratified split because we have largely imbalanced classes.

In [ ]:
birth_train, birth_test = train_test_split(birth.copy(),
                                          shuffle=True,
                                          random_state=650,
                                          stratify=birth['birth_class'],
                                          test_size=0.2)

## Baseline Model

Find proportion of response variable that falls into either category.

In [ ]:
normalized_value_counts = birth_train['birth_class'].value_counts(normalize=True)

# Display the normalized value counts
print("Normalized value counts:")
print(normalized_value_counts)

In `birth_train`, Term births are 86.3% and Preterm births are 13.7%.

In [ ]:
# Initialize empty lists to store evaluation metrics
baseline_precision = []
baseline_recalls = []
baseline_f1_scores = []
baseline_pr_aucs = []

# Perform 1000 random draws
for obs in range(1000):
    
    # Generate random binomial draws with probability 0.137
    draw = np.random.binomial(n=1, p=0.137, size=len(birth_train))
  
    # Calculate precision score and append to the list
    precision_obs = precision_score(birth_train.birth_class_binary, draw)
    baseline_precision.append(precision_obs)

    # Calculate recall score and append to the list
    recall_obs = recall_score(birth_train.birth_class_binary, draw)
    baseline_recalls.append(recall_obs)
    
    # Calculate precision-recall curve and AUC
    precision, recall, _ = precision_recall_curve(birth_train.birth_class_binary, draw)
    pr_auc = auc(recall, precision)
    baseline_pr_aucs.append(pr_auc)
    
    # Calculate F1 score and append to the list
    f1 = f1_score(birth_train.birth_class_binary, draw)
    baseline_f1_scores.append(f1)

plt.plot(recall,precision, marker='.', label='Logistic')
plt.xlabel('Recall')
plt.ylabel('Precision')

# Print statistics for recall, F1, and PR AUC scores
print("Here are some statistics for our baseline:")

print("Mean Precision - " + str(round(np.mean(baseline_precision), 6)))
print("Median Precision - " + str(round(np.median(baseline_precision), 6)))

print("Mean Recall - " + str(round(np.mean(baseline_recalls), 6)))
print("Median Recall - " + str(round(np.median(baseline_recalls), 6)))

print("Mean F1 Score - " + str(round(np.mean(baseline_f1_scores), 6)))
print("Median F1 Score - " + str(round(np.median(baseline_f1_scores), 6)))

print("Mean PR AUC - " + str(round(np.mean(baseline_pr_aucs), 6)))
print("Median PR AUC - " + str(round(np.median(baseline_pr_aucs), 6)))

## Pipeline to scale and one-hot encode all features in `birth_train`

In [ ]:
# One Hot Encoding of all categorical features

num_cols = ['maternal_age', 'BMI']

cat_cols = ['smoking_status', 'drinking_frequency', 'six_or_more_drinks_occurrence', 
                     'birth_order', 'diabetes', 'mental_health', 'drug_use']

preprocessor = ColumnTransformer(transformers=[('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols),
                                               ('num', StandardScaler(), num_cols)],
                                 remainder='passthrough')

birth_train_encoded = preprocessor.fit_transform(birth_train)

In [ ]:
# Convert the result back to a DataFrame for easier inspection
columns_after_encoding = preprocessor.get_feature_names_out()

# Custom function to remove prefixes
def remove_prefix(prefix, column_names):
    return [name[len(prefix):] if name.startswith(prefix) else name for name in column_names]

columns_after_encoding = remove_prefix("cat__", columns_after_encoding)
columns_after_encoding = remove_prefix("num__", columns_after_encoding)
columns_after_encoding = remove_prefix("remainder__", columns_after_encoding)

birth_train_encoded = pd.DataFrame(birth_train_encoded, columns=columns_after_encoding)

In [ ]:
birth_train_encoded.columns

Make sure all data types are integers as needed.

In [ ]:
cat_feat = ['smoking_status_Non-smoker', 
                   'smoking_status_Smoker', 
                   'smoking_status_Unknown', 
                   'drinking_frequency_2to3Weekly',
                   'drinking_frequency_2to4Monthly',
                   'drinking_frequency_4orMoreWeekly',
                   'drinking_frequency_Monthly',
                   'drinking_frequency_Skipped',
                   'six_or_more_drinks_occurrence_Daily',
                   'six_or_more_drinks_occurrence_LessThanMonthly',
                   'six_or_more_drinks_occurrence_Monthly',
                   'six_or_more_drinks_occurrence_Never',
                   'six_or_more_drinks_occurrence_Skipped',
                   'six_or_more_drinks_occurrence_Weekly',
                   'drug_use_No', 
                   'drug_use_Yes',
                   'birth_order_1',
                   'birth_order_2',
                   'birth_order_3',
                   'diabetes_None',
                   'diabetes_Type1',
                   'diabetes_Type2',
                   'diabetes_TypeUnknown',
                   'mental_health_Anxiety',
                   'mental_health_Depression',
                   'mental_health_None',
                   'mental_health_Other']

# Make sure all encoded variables are integer types
birth_train_encoded.birth_class_binary = birth_train_encoded.birth_class_binary.astype(int)
birth_train_encoded[cat_feat] = birth_train_encoded[cat_feat].astype(int)

In [ ]:
birth_train_encoded.sample(10)

## Logistic Regression with No Interactions, all features included

### Defining the model features

These include the following variables:

Continuous:
- Maternal age
- BMI

Categorical:
- Smoking Status
- Drinking frequency
- Binge drinking
- Drug use
- Diabetes
- Mental health
- Birth order

In [ ]:
model_feat = ['maternal_age', 'BMI']

# Add one-hot encoded lifestyle features
model_feat.extend(['smoking_status_Non-smoker', 
                   'smoking_status_Smoker', 
                   'smoking_status_Unknown', 
                   'drinking_frequency_2to3Weekly',
                   'drinking_frequency_2to4Monthly',
                   'drinking_frequency_4orMoreWeekly',
                   'drinking_frequency_Monthly',
                   'drinking_frequency_Skipped',
                   'six_or_more_drinks_occurrence_Daily',
                   'six_or_more_drinks_occurrence_LessThanMonthly',
                   'six_or_more_drinks_occurrence_Monthly',
                   'six_or_more_drinks_occurrence_Never',
                   'six_or_more_drinks_occurrence_Skipped',
                   'six_or_more_drinks_occurrence_Weekly',
                   'drug_use_No', 
                   'drug_use_Yes'])

# Add one-hot encoded health features
model_feat.extend(['birth_order_1',
                   'birth_order_2',
                   'birth_order_3',
                   'diabetes_None',
                   'diabetes_Type1',
                   'diabetes_Type2',
                   'diabetes_TypeUnknown',
                   'mental_health_Anxiety',
                   'mental_health_Depression',
                   'mental_health_None',
                   'mental_health_Other'])


### Create Stratified KFold Object

We will perform a stratified 10-fold cross-validation due to the imbalanced class sizes in the response variable.

In [ ]:
kfold = StratifiedKFold(n_splits=10,
                        shuffle=True,
                        random_state=123)

num_splits = 10

### Run Model with Cross Validation

In [ ]:
# Initialize arrays to store evaluation metrics
prec = np.zeros(num_splits)
recalls = np.zeros(num_splits)
f1 = np.zeros(num_splits)
pr_auc = np.zeros(num_splits)

# Start counter
counter = 0

# Loop through each KFold split
for train_index, test_index in kfold.split(birth_train_encoded, birth_train_encoded.birth_class_binary):
    birth_tt = birth_train_encoded.iloc[train_index]
    birth_ho = birth_train_encoded.iloc[test_index]

    log_reg = LogisticRegression(penalty='l2', class_weight='balanced', max_iter=1000)
        
    log_reg.fit(birth_tt[model_feat].values, birth_tt.birth_class_binary.values)
        
    pred = log_reg.predict(birth_ho[model_feat].values)
    
    prec[counter] = precision_score(birth_ho.birth_class_binary.values, pred)
    recalls[counter] = recall_score(birth_ho.birth_class_binary.values, pred)
    f1[counter] = f1_score(birth_ho.birth_class_binary.values, pred)
    
    # Calculate precision-recall curve and AUC
    precision, recall, _ = precision_recall_curve(birth_ho.birth_class_binary.values, log_reg.predict_proba(birth_ho[model_feat].values)[:,1])
    pr_auc[counter] = auc(recall, precision)
    
    # Calculate confusion matrix
    conf_mat = confusion_matrix(birth_ho.birth_class_binary.values, pred)
    
    print('---Metrics for Fold ', counter+1,'---')
    print('Precision: ', prec[counter])
    print('Recall: ', recalls[counter])
    print('F1: ', f1[counter])
    print('AUC: ', pr_auc[counter])
    print('Confusion Matrix: ')
    print(conf_mat)
    print(' ')
    
    plt.plot(recall,precision, marker='.', label='Logistic')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    
    #Adjust counter for next k-fold split
    counter += 1

In [ ]:
# Print statistics for recall, F1, and PR AUC scores
print("Here are the statistics for the recall score of our model:")

print("Mean Precision - " + str(round(np.mean(precision), 6)))
print("Median Precision - " + str(round(np.median(precision), 6)))

print("Mean Recall - " + str(round(np.mean(recalls), 6)))
print("Median Recall - " + str(round(np.median(recalls), 6)))

print("Mean F1 Score - " + str(round(np.mean(f1), 6)))
print("Median F1 Score - " + str(round(np.median(f1), 6)))

print("Mean PR AUC - " + str(round(np.mean(pr_auc), 6)))
print("Median PR AUC - " + str(round(np.median(pr_auc), 6)))

## Logistic Regression with No Interactions, selected features only

### Defining the model features

Based on EDA, we will select only features that gave a clear difference between preterm and term to include in our model.

These include the following variables:

Categorical:
- Drug use
- Diabetes

In [ ]:
model_feat = ['drug_use_No',
                'drug_use_Yes',
                'diabetes_None',
                'diabetes_Type1',
                'diabetes_Type2',
                'diabetes_TypeUnknown']

### Create Stratified KFold Object

We will perform a stratified 10-fold cross-validation due to the imbalanced class sizes in the response variable.

In [ ]:
kfold = StratifiedKFold(n_splits=10,
                        shuffle=True,
                        random_state=123)

num_splits = 10

### Run Model with Cross Validation

In [ ]:
# Initialize arrays to store evaluation metrics
prec = np.zeros(num_splits)
recalls = np.zeros(num_splits)
f1 = np.zeros(num_splits)
pr_auc = np.zeros(num_splits)

# Start counter
counter = 0

# Loop through each KFold split
for train_index, test_index in kfold.split(birth_train_encoded, birth_train_encoded.birth_class_binary):
    birth_tt = birth_train_encoded.iloc[train_index]
    birth_ho = birth_train_encoded.iloc[test_index]

    log_reg = LogisticRegression(penalty='l2', class_weight='balanced', max_iter=1000, random_state=456)
        
    log_reg.fit(birth_tt[model_feat].values, birth_tt.birth_class_binary.values)
        
    pred = log_reg.predict(birth_ho[model_feat].values)
    
    prec[counter] = precision_score(birth_ho.birth_class_binary.values, pred)
    recalls[counter] = recall_score(birth_ho.birth_class_binary.values, pred)
    f1[counter] = f1_score(birth_ho.birth_class_binary.values, pred)
    
    # Calculate precision-recall curve and AUC
    precision, recall, _ = precision_recall_curve(birth_ho.birth_class_binary.values, log_reg.predict_proba(birth_ho[model_feat].values)[:,1])
    pr_auc[counter] = auc(recall, precision)
    
    # Calculate confusion matrix
    conf_mat = confusion_matrix(birth_ho.birth_class_binary.values, pred)
    
    print('---Metrics for Fold ', counter+1,'---')
    print('Precision: ', prec[counter])
    print('Recall: ', recalls[counter])
    print('F1: ', f1[counter])
    print('AUC: ', pr_auc[counter])
    print('Confusion Matrix: ')
    print(conf_mat)
    print(' ')
    
    plt.plot(recall,precision, marker='.', label='Logistic')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    
    #Adjust counter for next k-fold split
    counter += 1

In [ ]:
# Print statistics for recall, F1, and PR AUC scores
print("Here are the statistics for the recall score of our model:")

print("Mean Precision - " + str(round(np.mean(precision), 6)))
print("Median Precision - " + str(round(np.median(precision), 6)))

print("Mean Recall - " + str(round(np.mean(recalls), 6)))
print("Median Recall - " + str(round(np.median(recalls), 6)))

print("Mean F1 Score - " + str(round(np.mean(f1), 6)))
print("Median F1 Score - " + str(round(np.median(f1), 6)))

print("Mean PR AUC - " + str(round(np.mean(pr_auc), 6)))
print("Median PR AUC - " + str(round(np.median(pr_auc), 6)))

## Logistic Regression with Interactions

### Defining the model features

We will only use interactions between continuous variables, because there are not enough observations in each of the categorical variables to include interaction terms.

These include the following variables:

Continuous:
- Maternal age
- BMI

Categorical:
- Smoking Status
- Drinking frequency
- Binge drinking
- Drug use
- Diabetes
- Mental health
- Birth order

Interactions: 
- Maternal age & BMI

Create interaction features and add to train and test data.

In [ ]:
# Specify which features for interaction
interaction_features = ['maternal_age', 'BMI']

# Create a pipeline with PolynomialFeatures
degree = 2  # Set the degree of interaction terms
poly = PolynomialFeatures(degree, interaction_only=True, include_bias=False)
birth_train_poly = poly.fit_transform(birth_train_encoded[interaction_features])

# Concatenate the polynomial features with the original features
birth_train = pd.concat([birth_train_encoded, pd.DataFrame(birth_train_poly, columns=poly.get_feature_names_out(interaction_features))], axis=1)
#birth_train = birth_train.drop(columns=birth_train.columns[[-2, -3]])

In [ ]:
birth_train.columns

Define all features for model:

In [ ]:
model_feat = ['maternal_age', 'BMI']

# Add one-hot encoded lifestyle features
model_feat.extend(['smoking_status_Non-smoker', 
                   'smoking_status_Smoker', 
                   'smoking_status_Unknown', 
                   'drinking_frequency_2to3Weekly',
                   'drinking_frequency_2to4Monthly',
                   'drinking_frequency_4orMoreWeekly',
                   'drinking_frequency_Monthly',
                   'drinking_frequency_Skipped',
                   'six_or_more_drinks_occurrence_Daily',
                   'six_or_more_drinks_occurrence_LessThanMonthly',
                   'six_or_more_drinks_occurrence_Monthly',
                   'six_or_more_drinks_occurrence_Never',
                   'six_or_more_drinks_occurrence_Skipped',
                   'six_or_more_drinks_occurrence_Weekly',
                   'drug_use_No', 
                   'drug_use_Yes'])

# Add one-hot encoded health features
model_feat.extend(['birth_order_1',
                   'birth_order_2',
                   'birth_order_3',
                   'diabetes_None',
                   'diabetes_Type1',
                   'diabetes_Type2',
                   'diabetes_TypeUnknown',
                   'mental_health_Anxiety',
                   'mental_health_Depression',
                   'mental_health_None',
                   'mental_health_Other'])

# Add interaction term
model_feat.extend(['maternal_age BMI'])

### Create Stratified KFold Object

We will perform a stratified 10-fold cross-validation due to the imbalanced class sizes in the response variable.

In [ ]:
kfold = StratifiedKFold(n_splits=10,
                        shuffle=True,
                        random_state=123)

num_splits = 10

### Run Model with Cross Validation

In [ ]:
# Initialize arrays to store evaluation metrics
prec = np.zeros(num_splits)
recalls = np.zeros(num_splits)
f1 = np.zeros(num_splits)
pr_auc = np.zeros(num_splits)

# Start counter
counter = 0

# Loop through each KFold split
for train_index, test_index in kfold.split(birth_train, birth_train.birth_class_binary):
    birth_tt = birth_train.iloc[train_index]
    birth_ho = birth_train.iloc[test_index]

    log_reg = LogisticRegression(penalty='l2', class_weight='balanced', max_iter=1000)
        
    log_reg.fit(birth_tt[model_feat].values, birth_tt.birth_class_binary.values)
        
    pred = log_reg.predict(birth_ho[model_feat].values)
    
    prec[counter] = precision_score(birth_ho.birth_class_binary.values, pred)
    recalls[counter] = recall_score(birth_ho.birth_class_binary.values, pred)
    f1[counter] = f1_score(birth_ho.birth_class_binary.values, pred)
    
    # Calculate precision-recall curve and AUC
    precision, recall, _ = precision_recall_curve(birth_ho.birth_class_binary.values, log_reg.predict_proba(birth_ho[model_feat].values)[:,1])
    pr_auc[counter] = auc(recall, precision)
    
    # Calculate confusion matrix
    conf_mat = confusion_matrix(birth_ho.birth_class_binary.values, pred)
    
    print('---Metrics for Fold ', counter+1,'---')
    print('Precision: ', prec[counter])
    print('Recall: ', recalls[counter])
    print('F1: ', f1[counter])
    print('AUC: ', pr_auc[counter])
    print('Confusion Matrix: ')
    print(conf_mat)
    print(' ')
    
    plt.plot(recall,precision, marker='.', label='Logistic')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    
    #Adjust counter for next k-fold split
    counter += 1

In [ ]:
# Print statistics for recall, F1, and PR AUC scores
print("Here are the statistics for the recall score of our model:")

print("Mean Precision - " + str(round(np.mean(precision), 6)))
print("Median Precision - " + str(round(np.median(precision), 6)))

print("Mean Recall - " + str(round(np.mean(recalls), 6)))
print("Median Recall - " + str(round(np.median(recalls), 6)))

print("Mean F1 Score - " + str(round(np.mean(f1), 6)))
print("Median F1 Score - " + str(round(np.median(f1), 6)))

print("Mean PR AUC - " + str(round(np.mean(pr_auc), 6)))
print("Median PR AUC - " + str(round(np.median(pr_auc), 6)))

## SVC

### Defining the model features

We will use the features that gave us the best metrics when using logistic regression.

These include the following variables:

Continuous:
- Maternal age
- BMI

Categorical:
- Smoking Status
- Drinking frequency
- Binge drinking
- Drug use
- Diabetes
- Mental health
- Birth order

In [ ]:
model_feat = ['maternal_age', 'BMI']

# Add one-hot encoded lifestyle features
model_feat.extend(['smoking_status_Non-smoker', 
                   'smoking_status_Smoker', 
                   'smoking_status_Unknown', 
                   'drinking_frequency_2to3Weekly',
                   'drinking_frequency_2to4Monthly',
                   'drinking_frequency_4orMoreWeekly',
                   'drinking_frequency_Monthly',
                   'drinking_frequency_Skipped',
                   'six_or_more_drinks_occurrence_Daily',
                   'six_or_more_drinks_occurrence_LessThanMonthly',
                   'six_or_more_drinks_occurrence_Monthly',
                   'six_or_more_drinks_occurrence_Never',
                   'six_or_more_drinks_occurrence_Skipped',
                   'six_or_more_drinks_occurrence_Weekly',
                   'drug_use_No', 
                   'drug_use_Yes'])

# Add one-hot encoded health features
model_feat.extend(['birth_order_1',
                   'birth_order_2',
                   'birth_order_3',
                   'diabetes_None',
                   'diabetes_Type1',
                   'diabetes_Type2',
                   'diabetes_TypeUnknown',
                   'mental_health_Anxiety',
                   'mental_health_Depression',
                   'mental_health_None',
                   'mental_health_Other'])


### Create Stratified KFold Object

We will perform a stratified 10-fold cross-validation due to the imbalanced class sizes in the response variable.

In [ ]:
kfold = StratifiedKFold(n_splits=10,
                        shuffle=True,
                        random_state=123)

num_splits = 10

### Run Model with Cross Validation

In [ ]:
# Define weight as inversely proportional to class frequencies (in SVC documentation)
weights = len(birth_train_encoded) / (2 * np.bincount(birth_train_encoded.birth_class_binary))

# Initialize arrays to store evaluation metrics
svc_prec = np.zeros(num_splits)
svc_recalls = np.zeros(num_splits)
svc_f1 = np.zeros(num_splits)
svc_pr_auc = np.zeros(num_splits)

# Initialize list to store predictions
svc_predictions = []

counter = 0

for train_index, test_index in kfold.split(birth_train_encoded, birth_train_encoded.birth_class_binary):
    birth_tt = birth_train_encoded.iloc[train_index]
    birth_ho = birth_train_encoded.iloc[test_index]

    # Create dictionary for class weights, with preterm birth having a heavier weight
    class_weights = {0: weights[0], 1: weights[1]}  

    # Add a column for class weights to birth_tt
    birth_tt = birth_tt.copy()
    birth_tt['class_weights'] = birth_tt['birth_class_binary'].map(class_weights)

    # Create the SVC
    svc = SVC(probability=True, random_state=404, class_weight='balanced')

    # Fit the pipeline on training data
    svc.fit(birth_tt[model_feat].values, birth_tt.birth_class_binary.values) 
    #        sample_weight=birth_tt.class_weights.values)

    # Predict on the holdout data
    svc_pred = svc.predict(birth_ho[model_feat].values)

    # Append predictions to the list
    svc_predictions.append(svc_pred)

    # Calculate evaluation metrics
    svc_prec[counter] = precision_score(birth_ho.birth_class_binary.values, svc_pred)
    svc_recalls[counter] = recall_score(birth_ho.birth_class_binary.values, svc_pred)
    svc_f1[counter] = f1_score(birth_ho.birth_class_binary.values, svc_pred)

    # Calculate precision-recall curve and AUC
    svc_precision, svc_recall, _ = precision_recall_curve(birth_ho.birth_class_binary, 
                                                          svc.predict_proba(birth_ho[model_feat])[:, 1])
    svc_pr_auc[counter] = auc(svc_recall, svc_precision)

    # Calculate confusion matrix
    conf_mat = confusion_matrix(birth_ho.birth_class_binary.values, svc_pred)

    # Print all metrics
    print('---Metrics for Fold ', counter+1,'---')
    print('Precision: ', svc_prec[counter])
    print('Recall: ', svc_recalls[counter])
    print('F1: ', svc_f1[counter])
    print('AUC: ', svc_pr_auc[counter])
    print('Confusion Matrix: ')
    print(conf_mat)
    print(' ')
    
    plt.plot(svc_recall, svc_precision, marker='.', label='SVC')
    plt.xlabel('Recall')
    plt.ylabel('Precision')

    # Adjust counter for the next k-fold split
    counter += 1


In [ ]:
# Print statistics for recall, F1, and PR AUC scores
print("Here are the statistics for the recall score of our model:")

print("Mean Precision - " + str(round(np.mean(svc_prec), 6)))
print("Median Precision - " + str(round(np.median(svc_prec), 6)))

print("Mean Recall - " + str(round(np.mean(svc_recalls), 6)))
print("Median Recall - " + str(round(np.median(svc_recalls), 6)))

print("Mean F1 Score - " + str(round(np.mean(svc_f1), 6)))
print("Median F1 Score - " + str(round(np.median(svc_f1), 6)))

print("Mean PR AUC - " + str(round(np.mean(svc_pr_auc), 6)))
print("Median PR AUC - " + str(round(np.median(svc_pr_auc), 6)))

## ROSE Method for Unbalanced Data

Install imbalanced learn package and import libraries:

In [ ]:
# Only need to do this once
!pip install -U imbalanced-learn

In [ ]:
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import KFold

### use RandomOverSampler to oversample minority class

In [ ]:
print('Original dataset: ')
print(birth_train_encoded.birth_class_binary.value_counts())
print('')

ros = RandomOverSampler(random_state=42)

birth_train_rose, birth_train_rose.birth_class_binary = ros.fit_resample(birth_train_encoded, birth_train_encoded.birth_class_binary)

print('Oversampled dataset: ')
print(birth_train_rose.birth_class_binary.value_counts())

### Run a new baseline model, with oversampled minority class data

Find proportion of response variable that falls into either category.

In [ ]:
normalized_value_counts = birth_train_rose['birth_class'].value_counts(normalize=True)

# Display the normalized value counts
print("Normalized value counts:")
print(normalized_value_counts)

In [ ]:
# Initialize empty lists to store evaluation metrics
baseline_precision = []
baseline_recalls = []
baseline_f1_scores = []
baseline_pr_aucs = []

# Perform 1000 random draws
for obs in range(1000):
    
    # Generate random binomial draws with probability 0.5
    draw = np.random.binomial(n=1, p=0.5, size=len(birth_train_rose))
  
    # Calculate precision score and append to the list
    precision_obs = precision_score(birth_train_rose.birth_class_binary, draw)
    baseline_precision.append(precision_obs)

    # Calculate recall score and append to the list
    recall_obs = recall_score(birth_train_rose.birth_class_binary, draw)
    baseline_recalls.append(recall_obs)
    
    # Calculate precision-recall curve and AUC
    precision, recall, _ = precision_recall_curve(birth_train_rose.birth_class_binary, draw)
    pr_auc = auc(recall, precision)
    baseline_pr_aucs.append(pr_auc)
    
    # Calculate F1 score and append to the list
    f1 = f1_score(birth_train_rose.birth_class_binary, draw)
    baseline_f1_scores.append(f1)

plt.plot(recall,precision, marker='.', label='Logistic')
plt.xlabel('Recall')
plt.ylabel('Precision')

# Print statistics for recall, F1, and PR AUC scores
print("Here are some statistics for our baseline:")

print("Mean Precision - " + str(round(np.mean(baseline_precision), 6)))
print("Median Precision - " + str(round(np.median(baseline_precision), 6)))

print("Mean Recall - " + str(round(np.mean(baseline_recalls), 6)))
print("Median Recall - " + str(round(np.median(baseline_recalls), 6)))

print("Mean F1 Score - " + str(round(np.mean(baseline_f1_scores), 6)))
print("Median F1 Score - " + str(round(np.median(baseline_f1_scores), 6)))

print("Mean PR AUC - " + str(round(np.mean(baseline_pr_aucs), 6)))
print("Median PR AUC - " + str(round(np.median(baseline_pr_aucs), 6)))

### Defining the model features

We will use the features that gave us the best metrics when using logistic regression.

These include the following variables:

Continuous:
- Maternal age
- BMI

Categorical:
- Smoking Status
- Drinking frequency
- Binge drinking
- Drug use
- Diabetes
- Mental health
- Birth order

In [ ]:
model_feat = ['maternal_age', 'BMI']

# Add one-hot encoded lifestyle features
model_feat.extend(['smoking_status_Non-smoker', 
                   'smoking_status_Smoker', 
                   'smoking_status_Unknown', 
                   'drinking_frequency_2to3Weekly',
                   'drinking_frequency_2to4Monthly',
                   'drinking_frequency_4orMoreWeekly',
                   'drinking_frequency_Monthly',
                   'drinking_frequency_Skipped',
                   'six_or_more_drinks_occurrence_Daily',
                   'six_or_more_drinks_occurrence_LessThanMonthly',
                   'six_or_more_drinks_occurrence_Monthly',
                   'six_or_more_drinks_occurrence_Never',
                   'six_or_more_drinks_occurrence_Skipped',
                   'six_or_more_drinks_occurrence_Weekly',
                   'drug_use_No', 
                   'drug_use_Yes'])

# Add one-hot encoded health features
model_feat.extend(['birth_order_1',
                   'birth_order_2',
                   'birth_order_3',
                   'diabetes_None',
                   'diabetes_Type1',
                   'diabetes_Type2',
                   'diabetes_TypeUnknown',
                   'mental_health_Anxiety',
                   'mental_health_Depression',
                   'mental_health_None',
                   'mental_health_Other'])

### Create Stratified KFold Object

We will perform a stratified 10-fold cross-validation due to the imbalanced class sizes in the response variable.

In [ ]:
kfold = KFold(n_splits=10, shuffle=True, random_state=123)

num_splits = 10

### Run Model with Cross Validation

In [ ]:
# Initialize arrays to store evaluation metrics
prec = np.zeros(num_splits)
recalls = np.zeros(num_splits)
f1 = np.zeros(num_splits)
pr_auc = np.zeros(num_splits)

# Start counter
counter = 0

# Loop through each KFold split
for train_index, test_index in kfold.split(birth_train_rose, birth_train_rose.birth_class_binary):
    birth_tt = birth_train_rose.iloc[train_index]
    birth_ho = birth_train_rose.iloc[test_index]

    log_reg = LogisticRegression(max_iter=1000)
        
    log_reg.fit(birth_tt[model_feat].values, birth_tt.birth_class_binary.values)
        
    pred = log_reg.predict(birth_ho[model_feat].values)
    
    prec[counter] = precision_score(birth_ho.birth_class_binary.values, pred)
    recalls[counter] = recall_score(birth_ho.birth_class_binary.values, pred)
    f1[counter] = f1_score(birth_ho.birth_class_binary.values, pred)
    
    # Calculate precision-recall curve and AUC
    precision, recall, _ = precision_recall_curve(birth_ho.birth_class_binary.values, log_reg.predict_proba(birth_ho[model_feat].values)[:,1])
    pr_auc[counter] = auc(recall, precision)
    
    # Calculate confusion matrix
    conf_mat = confusion_matrix(birth_ho.birth_class_binary.values, pred)
    
    print('---Metrics for Fold ', counter+1,'---')
    print('Precision: ', prec[counter])
    print('Recall: ', recalls[counter])
    print('F1: ', f1[counter])
    print('AUC: ', pr_auc[counter])
    print('Confusion Matrix: ')
    print(conf_mat)
    print(' ')
    
    plt.plot(recall,precision, marker='.', label='Logistic')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    
    #Adjust counter for next k-fold split
    counter += 1

In [ ]:
# Print statistics for recall, F1, and PR AUC scores
print("Here are the statistics for the recall score of our model:")

print("Mean Precision - " + str(round(np.mean(precision), 6)))
print("Median Precision - " + str(round(np.median(precision), 6)))

print("Mean Recall - " + str(round(np.mean(recalls), 6)))
print("Median Recall - " + str(round(np.median(recalls), 6)))

print("Mean F1 Score - " + str(round(np.mean(f1), 6)))
print("Median F1 Score - " + str(round(np.median(f1), 6)))

print("Mean PR AUC - " + str(round(np.mean(pr_auc), 6)))
print("Median PR AUC - " + str(round(np.median(pr_auc), 6)))

# Run Chosen Model on Test Data

## Pipeline to scale and one-hot encode all features in `birth_test`

In [ ]:
# One Hot Encoding of all categorical features

num_cols = ['maternal_age', 'BMI']

cat_cols = ['smoking_status', 'drinking_frequency', 'six_or_more_drinks_occurrence', 
                     'birth_order', 'diabetes', 'mental_health', 'drug_use']

preprocessor = ColumnTransformer(transformers=[('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols),
                                               ('num', StandardScaler(), num_cols)],
                                 remainder='passthrough')

birth_test_encoded = preprocessor.fit_transform(birth_test)

In [ ]:
# Convert the result back to a DataFrame for easier inspection
columns_after_encoding = preprocessor.get_feature_names_out()

# Custom function to remove prefixes
def remove_prefix(prefix, column_names):
    return [name[len(prefix):] if name.startswith(prefix) else name for name in column_names]

columns_after_encoding = remove_prefix("cat__", columns_after_encoding)
columns_after_encoding = remove_prefix("num__", columns_after_encoding)
columns_after_encoding = remove_prefix("remainder__", columns_after_encoding)

birth_test_encoded = pd.DataFrame(birth_test_encoded, columns=columns_after_encoding)

In [ ]:
birth_test_encoded.columns

Make sure all data types are integers as needed.

In [ ]:
cat_feat = ['smoking_status_Non-smoker', 
                   'smoking_status_Smoker', 
                   'smoking_status_Unknown', 
                   'drinking_frequency_2to3Weekly',
                   'drinking_frequency_2to4Monthly',
                   'drinking_frequency_4orMoreWeekly',
                   'drinking_frequency_Monthly',
                   'drinking_frequency_Skipped',
                   'six_or_more_drinks_occurrence_Daily',
                   'six_or_more_drinks_occurrence_LessThanMonthly',
                   'six_or_more_drinks_occurrence_Monthly',
                   'six_or_more_drinks_occurrence_Never',
                   'six_or_more_drinks_occurrence_Skipped',
                   'six_or_more_drinks_occurrence_Weekly',
                   'drug_use_No', 
                   'drug_use_Yes',
                   'birth_order_1',
                   'birth_order_2',
                   'birth_order_3',
                   'diabetes_None',
                   'diabetes_Type1',
                   'diabetes_Type2',
                   'diabetes_TypeUnknown',
                   'mental_health_Anxiety',
                   'mental_health_Depression',
                   'mental_health_None',
                   'mental_health_Other']

# Make sure all encoded variables are integer types
birth_test_encoded.birth_class_binary = birth_test_encoded.birth_class_binary.astype(int)
birth_test_encoded[cat_feat] = birth_test_encoded[cat_feat].astype(int)

In [ ]:
birth_test_encoded.sample(10)

## Run Baseline Model on Test Data

Find proportion of response variable that falls into either category.

In [ ]:
normalized_value_counts = birth_test['birth_class'].value_counts(normalize=True)

# Display the normalized value counts
print("Normalized value counts:")
print(normalized_value_counts)

In [ ]:
birth['birth_class'].value_counts()

In [ ]:
7568/1203

In `birth_test`, Term births are 86.3% and Preterm births are 13.7%.

In [ ]:

# Initialize empty lists to store evaluation metrics
baseline_precision = []
baseline_recalls = []
baseline_f1_scores = []
baseline_pr_aucs = []

# Perform 1000 random draws
for obs in range(1000):
    
    # Generate random binomial draws with probability 0.137
    draw = np.random.binomial(n=1, p=0.137, size=len(birth_test))
  
    # Calculate precision score and append to the list
    precision_obs = precision_score(birth_test.birth_class_binary, draw)
    baseline_precision.append(precision_obs)

    # Calculate recall score and append to the list
    recall_obs = recall_score(birth_test.birth_class_binary, draw)
    baseline_recalls.append(recall_obs)
    
    # Calculate precision-recall curve and AUC
    base_precision, base_recall, _ = precision_recall_curve(birth_test.birth_class_binary, draw)
    pr_auc = auc(base_recall, base_precision)
    baseline_pr_aucs.append(pr_auc)
    
    # Calculate F1 score and append to the list
    f1 = f1_score(birth_test.birth_class_binary, draw)
    baseline_f1_scores.append(f1)

plt.plot(base_recall,base_precision, marker='.')
plt.xlabel('Recall')
plt.ylabel('Precision')

# Print statistics for recall, F1, and PR AUC scores
print("Here are some statistics for our baseline:")

print("Mean Precision - " + str(round(np.mean(baseline_precision), 6)))
print("Median Precision - " + str(round(np.median(baseline_precision), 6)))

print("Mean Recall - " + str(round(np.mean(baseline_recalls), 6)))
print("Median Recall - " + str(round(np.median(baseline_recalls), 6)))

print("Mean F1 Score - " + str(round(np.mean(baseline_f1_scores), 6)))
print("Median F1 Score - " + str(round(np.median(baseline_f1_scores), 6)))

print("Mean PR AUC - " + str(round(np.mean(baseline_pr_aucs), 6)))
print("Median PR AUC - " + str(round(np.median(baseline_pr_aucs), 6)))

## Run Best Model — Logistic Regression with Class Weights

### Defining the model features

These include the following variables:

Continuous:
- Maternal age
- BMI

Categorical:
- Smoking Status
- Drinking frequency
- Binge drinking
- Drug use
- Diabetes
- Mental health
- Birth order

In [ ]:
model_feat = ['maternal_age', 'BMI']

# Add one-hot encoded lifestyle features
model_feat.extend(['smoking_status_Non-smoker', 
                   'smoking_status_Smoker', 
                   'smoking_status_Unknown', 
                   'drinking_frequency_2to3Weekly',
                   'drinking_frequency_2to4Monthly',
                   'drinking_frequency_4orMoreWeekly',
                   'drinking_frequency_Monthly',
                   'drinking_frequency_Skipped',
                   'six_or_more_drinks_occurrence_Daily',
                   'six_or_more_drinks_occurrence_LessThanMonthly',
                   'six_or_more_drinks_occurrence_Monthly',
                   'six_or_more_drinks_occurrence_Never',
                   'six_or_more_drinks_occurrence_Skipped',
                   'six_or_more_drinks_occurrence_Weekly',
                   'drug_use_No', 
                   'drug_use_Yes'])

# Add one-hot encoded health features
model_feat.extend(['birth_order_1',
                   'birth_order_2',
                   'birth_order_3',
                   'diabetes_None',
                   'diabetes_Type1',
                   'diabetes_Type2',
                   'diabetes_TypeUnknown',
                   'mental_health_Anxiety',
                   'mental_health_Depression',
                   'mental_health_None',
                   'mental_health_Other'])

### Run Final Model and Print Metrics

In [ ]:
# # Initialize arrays to store evaluation metrics
# prec = np.zeros(num_splits)
# recalls = np.zeros(num_splits)
# f1 = np.zeros(num_splits)
# pr_auc = np.zeros(num_splits)

# # Start counter
# counter = 0

# Loop through each KFold split
# for train_index, test_index in kfold.split(birth_train_encoded, birth_train_encoded.birth_class_binary):
#     birth_tt = birth_train_encoded.iloc[train_index]
#     birth_ho = birth_train_encoded.iloc[test_index]

log_reg = LogisticRegression(penalty='l2', class_weight='balanced', max_iter=1000)

log_reg.fit(birth_train_encoded[model_feat].values, birth_train_encoded.birth_class_binary.values)

pred = log_reg.predict(birth_test_encoded[model_feat].values)

precision = precision_score(birth_test_encoded.birth_class_binary.values, pred)
recall = recall_score(birth_test_encoded.birth_class_binary.values, pred)
f1 = f1_score(birth_test_encoded.birth_class_binary.values, pred)

# Calculate precision-recall curve and AUC
precisions, recalls, _ = precision_recall_curve(birth_test_encoded.birth_class_binary.values, log_reg.predict_proba(birth_test_encoded[model_feat].values)[:,1])
pr_auc = auc(recalls, precisions)

# Calculate confusion matrix
conf_mat = confusion_matrix(birth_test_encoded.birth_class_binary.values, pred)

print('--- Final Model Metrics on Test Data ---')
print('Precision: ', precision)
print('Recall: ', recall)
print('F1: ', f1)
print('AUC: ', pr_auc)
print('Confusion Matrix: ')
print(conf_mat)
print(' ')

plt.plot(recalls,precisions, marker='.', label='Logistic')
plt.xlabel('Recall')
plt.ylabel('Precision')

#Adjust counter for next k-fold split
#counter += 1

## Make plot of model PR-AUC vs baseline PR-AUC

In [ ]:
plt.figure(figsize=(10,5))

# Plot baseline
plt.plot(base_recall, base_precision, linestyle='dashed', marker='.', label='Baseline Model')

# Plot model
plt.plot(recalls, precisions, marker='.', label='Final Model')

plt.title('Precision-Recall Curves for Baseline and Final Models', fontsize="20")
plt.xlabel('Recall', fontsize="15")
plt.ylabel('Precision', fontsize="15")
plt.legend(fontsize="15", loc ="upper right")

plt.show()

## Fairness Metrics

### One-hot encoding for fairness metrics because I couldn't get the pipeline to work

#### Birth_encoded - entire birth dataframe for metrics on entire dataset

In [ ]:
birth_encoded = pd.get_dummies(birth, columns=['race_person', 'ethnicity_person'], prefix=['race', 'ethnicity'], prefix_sep='_')

# Drop datetime and obj col as it causes errors and is not necessary
columns_to_drop = ['condition_start_date', 'birth_class', 'date_of_birth']
birth_encoded = birth_encoded.drop(columns=columns_to_drop)


birth_encoded = birth_encoded.select_dtypes(exclude=['object'])

birth_encoded.head()

#### Birth_ho encoding (birth holdout set) 

In [ ]:
birth_ho_encoded = pd.get_dummies(birth_ho, columns=['race_person', 'ethnicity_person'], prefix=['race', 'ethnicity'], prefix_sep='_')

columns_to_drop = ['condition_start_date', 'birth_class', 'date_of_birth']
birth_ho_encoded = birth_ho_encoded.drop(columns=columns_to_drop)
birth_ho_encoded.head()

#### Birth_test_data (Test dataset)

In [ ]:
birth_test_data_encoded = pd.get_dummies(birth_test_encoded, columns=['race_person', 'ethnicity_person'], prefix=['race', 'ethnicity'], prefix_sep='_')

columns_to_drop = ['condition_start_date', 'birth_class', 'date_of_birth']
birth_test_data_encoded = birth_test_data_encoded.drop(columns=columns_to_drop)
birth_test_data_encoded.head()

##### Calculate SPD and Equalized Odds on Training Data Holdout set using loop to access folds

In [ ]:
!pip install aif360

In [ ]:
import aif360
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.datasets import BinaryLabelDataset
from sklearn.metrics import accuracy_score
import numpy as np

# Assuming total_folds is the number of folds from your previous code
total_folds = kfold.get_n_splits()

race_categories = ['race_Black or African American', 'race_None of these', 'race_no answer',
                   'race_More than one population', 'race_Asian', 'race_Middle Eastern or North African',
                   'race_Native Hawaiian or Other Pacific Islander']

# Initialize arrays to store fairness metrics across folds for each minority group
mean_diffs = {group: [] for group in race_categories}
equalized_odds_ratios = {group: [] for group in race_categories}

# Iterate over folds
for fold in range(total_folds):
    
    # Use the predictions from the corresponding fold
    svc_pred_fold = svc_predictions[fold]

    # Iterate over minority groups
    for minority_group in race_categories:
        
        # Create a new BinaryLabelDataset for each minority group within the fold
        label_column_name = 'birth_class_binary'
        bld = BinaryLabelDataset(
            favorable_label=0, unfavorable_label=1,
            df=birth_ho_encoded, label_names=[label_column_name],
            protected_attribute_names=['race_White'] + race_categories
        )

        privileged_groups = [{'race_White': 1}]
        unprivileged_groups = [{'race_White': 0, minority_group: 1}]

        # Create an instance of BinaryLabelDatasetMetric
        metric_bld = BinaryLabelDatasetMetric(bld, privileged_groups=privileged_groups, unprivileged_groups=unprivileged_groups)

        # Calculate mean difference
        mean_diffs[minority_group].append(metric_bld.mean_difference())

        # Assuming you have ground truth labels and predicted labels
        cm = ClassificationMetric(bld, bld, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)

        # Calculate equalized odds ratio
        equalized_odds_ratios[minority_group].append(cm.average_odds_difference())

# Print the mean fairness metrics for each minority group across all folds
for minority_group in race_categories:
    print(f" Mean Difference ({minority_group}): {np.mean(mean_diffs[minority_group]).round(3)}")
    print(f" Equalized Odds Ratio ({minority_group}): {np.mean(equalized_odds_ratios[minority_group]).round(3)}")


### Fairness Metrics SPD and Equalized Odds on Test Predictions

In [ ]:
import aif360
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.datasets import BinaryLabelDataset
from sklearn.metrics import accuracy_score
import numpy as np



race_categories = ['race_Black or African American', 'race_None of these', 'race_no answer',
                   'race_More than one population', 'race_Asian', 'race_Middle Eastern or North African',
                   'race_Native Hawaiian or Other Pacific Islander']

# Initialize arrays to store fairness metrics across folds for each minority group
mean_diffs = {group: [] for group in race_categories}
equalized_odds_ratios = {group: [] for group in race_categories}


# Iterate over minority groups
for minority_group in race_categories:

    # Create a new BinaryLabelDataset for each minority group
    label_column_name = 'birth_class_binary'
    bld = BinaryLabelDataset(
        favorable_label=0, unfavorable_label=1,
        df=birth_test_data_encoded, label_names=[label_column_name],
        protected_attribute_names=['race_White'] + race_categories
    )

    privileged_groups = [{'race_White': 1}]
    unprivileged_groups = [{'race_White': 0, minority_group: 1}]

    # Create an instance of BinaryLabelDatasetMetric
    metric_bld = BinaryLabelDatasetMetric(bld, privileged_groups=privileged_groups, unprivileged_groups=unprivileged_groups)

    # Calculate mean difference
    mean_diffs[minority_group].append(metric_bld.mean_difference())

    # Assuming you have ground truth labels and predicted labels
    cm = ClassificationMetric(bld, bld, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)

    # Calculate equalized odds ratio
    equalized_odds_ratios[minority_group].append(cm.average_odds_difference())

# Print the mean fairness metrics for each minority group across all folds
for minority_group in race_categories:
    print(f" Mean Difference ({minority_group}): {mean_diffs[minority_group]}")
    print(f" Equalized Odds Ratio ({minority_group}): {equalized_odds_ratios[minority_group]}")


#### Ground truth SPD calculation (this applies to entire dataset to see what our actual disparities are); no pipeline

In [ ]:
import numpy as np
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.datasets import BinaryLabelDataset

for minority_group in race_categories:
    

    # Create a BinaryLabelDataset for ground truth labels
    label_column_name = 'birth_class_binary'
    bld_ground_truth = BinaryLabelDataset(
        favorable_label=0, unfavorable_label=1,
        df=birth_encoded, label_names=[label_column_name],
        protected_attribute_names=['race_White', minority_group]
    )

    # Set privileged and unprivileged groups
    privileged_groups = [{'race_White': 1}]
    unprivileged_groups = [{'race_White': 0, minority_group: 1}]

    # Create an instance of BinaryLabelDatasetMetric
    metric_bld_ground_truth = BinaryLabelDatasetMetric(bld_ground_truth, privileged_groups=privileged_groups, unprivileged_groups=unprivileged_groups)

    # Calculate SPD for ground truth labels
    spd_ground_truth = metric_bld_ground_truth.statistical_parity_difference()
              

    print(f" Statistical Parity Difference (Ground Truth):", minority_group, spd_ground_truth)
   

    